### Article Predictions 

#### We made a article recommendation system to predict the articles based on user intrests which is given in the form of prompt

In [7]:
import pandas as pd

main=pd.read_csv('cleaned_text.csv')
main.head()

,item_title,publication_title,item_doi,authors,publication_year,url,keywords
0,correction strong field physics pursued with p...,aapps bulletin,10.1007/s43673-021-00009-0,vishwa bandhu pathakseong ku leeki hong paecal...,2021,http://link.springer.com/article/10.1007/s4367...,AAPPS Bulletin Atomic Molecular Optical and Pl...
1,time reversal and reciprocity,aapps bulletin,10.1007/s43673-022-00060-5,olivier sigwarthchristian miniatura,2022,http://link.springer.com/article/10.1007/s4367...,AAPPS Bulletin Atomic Molecular Optical and Pl...
2,ultrasound detection using microcavity raman l...,aapps bulletin,10.1007/s43673-022-00068-x,xiao,2022,http://link.springer.com/article/10.1007/s4367...,AAPPS Bulletin Atomic Molecular Optical and Pl...
3,relativistic density functional theory in nucl...,aapps bulletin,10.1007/s43673-021-00001-8,jie mengpengwei zhao,2021,http://link.springer.com/article/10.1007/s4367...,AAPPS Bulletin Atomic Molecular Optical and Pl...
4,muon cooling and acceleration,aapps bulletin,10.1007/s43673-022-00035-6,masashi otani,2022,http://link.springer.com/article/10.1007/s4367...,AAPPS Bulletin Atomic Molecular Optical and Pl...


In [8]:
main.shape

(275755, 7)

### Journal Dataframe

#### Journal articles, keywords, authors concating them

In [9]:
journal_art = main.groupby('Publication Title')['Item Title'].apply(list).reset_index(name='Articles')
journal_art.set_index(['Publication Title'],inplace=True)
journal_auth = main.groupby('Publication Title')['Authors'].apply(list).reset_index(name='Authors')
journal_auth.set_index(['Publication Title'],inplace=True)
journal_key= main.drop_duplicates(subset=["Publication Title", "Keywords"], keep='first')
journal_key=journal_key.drop(['Item Title','Authors','Publication Year','URL'],axis=1)
journal_key.set_index(['Publication Title'],inplace=True)
journal_main = pd.concat([journal_key, journal_art,journal_auth], axis=1, join='inner')
journal_main=journal_main.reset_index()
journal_main.head()

KeyError: 'Publication Title'

#### Changing into paragraphs from lists

##### get paragraph:
1. combines joins the list of the data into a paragraph
##### get cleaned_text:
1. check if the instance is string
2. re.match(r'^[a-zA-Z]+$', word) checks if the word consists only of alphabetic characters
3. word not in stop_words checks if the word is not in a list of stop words.
4. len(word) > 1 checks if the word has a length greater than 1.
5. word[1] != '.' checks if the second character of the word is not a dot (.) character.
##### combine text:
1. combines all the indices required in the list of indices into a paragraph

In [17]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def get_paragraph(row,index):
  ans=''
  for x in row[index]:
    ans=ans+'  '+x.lower()
  return ans
def get_clean_text(row,index):
    if not isinstance(row[index], str):
        return '' 
    clean_text=''
    words = re.findall(r'\b\w+\b', row[index].lower())
    words = row[index].lower().split()
    for word in words:
        if( re.match(r'^[a-zA-Z]+$', word) and  word not in stop_words and len(word) >1 and word[1] != '.'):
            clean_text=clean_text+' '+word
    return clean_text
def combine(row,indices):
    ans=''
    for i in indices:
      ans=ans+' '+row[i]
    return ans

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vukya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
journal_main['Articles']=journal_main.apply(get_paragraph,index='Articles',axis=1)
journal_main['Articles']=journal_main.apply(get_clean_text,index='Articles',axis=1)
journal_main['Authors']=journal_main.apply(get_paragraph,index='Authors',axis=1)
journal_main['Authors']=journal_main.apply(get_clean_text,index='Authors',axis=1)
journal_main['Keywords']=journal_main.apply(get_clean_text,index=1,axis=1)
journal_main.head()

,Publication Title,Keywords,Articles,Authors
0,AAPPS Bulletin,aapps bulletin atomic molecular optical plasm...,correction strong field physics pursued petaw...,vishwa bandhu pathakseong ku leeki hong paeca...
1,AAPS PharmSciTech,aaps pharmscitech pharmacologytoxicology biot...,environmental monitoring closed robotic workc...,joseph mccallnonita barnardkevin gadientchand...
2,AI & SOCIETY,,correction dismantling chinese room linguisti...,lawrence lengbeyer clare foster francesca fof...
3,AI and Ethics,,publisher convergence source control actual a...,haleh asgarinia jakob floridi robert hannaemr...
4,AI in Civil Engineering,,ai civil engineering fusion thermal rgb image...,xianzhong zhao quincy alexandervedhus hoskere...


### Feature Extraction taking nouns and adjectives as features

#### Preprocessing the text

It is possible to predict journals using only nouns and adjectives as features. By focusing on nouns and adjectives, we can capture key information and characteristics of the articles that may be relevant for predicting the corresponding journals.

In [19]:
journal_main['Tokenized'] = journal_main['Articles'].apply(word_tokenize)
journal_main['Tagged'] = journal_main['Tokenized'].apply(pos_tag)

In [6]:
journal_main['Tags'] = journal_main['Tagged'].apply(lambda x: [word for word, tag in x if tag.startswith('NN') or tag.startswith('JJ')])
journal_main['Tags'] = journal_main['Tags'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
journal_main['Tags']=journal_main.apply(get_paragraph,index='Tags',axis=1)
journal_main=journal_main.drop(['Articles','Tokenized','Tagged'],axis=1)
journal_main['Tags']=journal_main.apply(combine,indices=['Keywords','Tags','Authors'],axis=1)
journal_main['Tags']=journal_main.apply(get_clean_text,index='Tags',axis=1)
journal_main=journal_main.drop(['Keywords','Authors'],axis=1)
journal_main.head()

,Publication Title,Tags
0,AAPPS Bulletin,aapps bulletin atomic molecular optical plasm...
1,AAPS PharmSciTech,aaps pharmscitech pharmacologytoxicology biot...
2,AI & SOCIETY,correction chinese room linguistic framework ...
3,AI and Ethics,publisher convergence source control actual a...
4,AI in Civil Engineering,civil engineering fusion thermal rgb images d...


### TFIDF vector for journal prediction 

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(decode_error='ignore',strip_accents='ascii')
journal_tfidf_matrix = vectorizer.fit_transform(journal_main['Tags'])

we predict based on cosine similarities

In [8]:
journal_threshold=4

In [9]:
def get_journal_index(user_input):
    user_tfidf = vectorizer.transform([user_input])
    cosine_similarities = cosine_similarity(user_tfidf, journal_tfidf_matrix).flatten()
    indices = cosine_similarities.argsort()[::-1]
    top_recommendations = [i for i in indices if cosine_similarities[i] > 0][:min(journal_threshold, len(indices))]
    return top_recommendations

#### Preparing individual article dataframes

In [10]:
def get_article_df(row):
    article = main.loc[main['Publication Title'] == journal_main['Publication Title'][row.name]].copy()
    article['Item Title']=article.apply(get_clean_text,index='Item Title',axis=1)
    article['Authors']=article.apply(get_clean_text,index='Authors',axis=1)
    article['Tokenized'] = article['Item Title'].apply(word_tokenize)
    article['Tagged'] = article['Tokenized'].apply(pos_tag)
    article['Tags'] = article['Tagged'].apply(lambda x: [word for word, tag in x if tag.startswith('NN') or tag.startswith('JJ')and word.lower() not in stop_words])
    article['Tags']=article.apply(get_paragraph,index='Tags',axis=1)
    article['Tags']=article.apply(lambda x : x['Tags']+' '+x['Authors']+' '+str(x['Publication Year']),axis=1)
    article=article.drop(['Keywords','Publication Title','Tokenized','Tagged','Authors','Publication Year'],axis=1)
    article.reset_index(inplace=True)
    article.set_index('index', inplace=True)
    return article

journal_main['article_df']=journal_main.apply(get_article_df,axis=1)

#### Generating tfidf matrices for the journals

In [11]:
def get_vectorizer(row):
    vectorizer = TfidfVectorizer(decode_error='ignore',strip_accents='ascii')
    return vectorizer
def get_tfidf_matrix(row):
    tfidf_matrix = row['article_vectorizer'].fit_transform(row['article_df']['Tags'])
    return tfidf_matrix
journal_main['article_vectorizer']=journal_main.apply(get_vectorizer,axis=1)
journal_main['article_matrix']=journal_main.apply(get_tfidf_matrix,axis=1)

In [12]:
article_threshold=10

#### Article recommendation using user input

In [13]:
def get_article_recommendations(user_input):
    recommended_journals=get_journal_index(user_input)
    l=[]
    for journal_id in recommended_journals:
        user_tfidf=journal_main['article_vectorizer'][journal_id].transform([user_input])
        cosine_similarities = cosine_similarity(user_tfidf, journal_main['article_matrix'][journal_id]).flatten()
        indices = cosine_similarities.argsort()[::-1]
        top_recommendation_articles = [(cosine_similarities[i],i,journal_id) for i in indices if cosine_similarities[i] > 0][:min(article_threshold, len(indices))]
        l=l+top_recommendation_articles
    l.sort(reverse=True)
    return l


In [14]:

def get_links(user_input):
    l=[]
    recommendation_list=get_article_recommendations(user_input) #You will get article id and journal id from here
    for article in recommendation_list:
        cosine_similarity,article_id,journal_id=article
        l.append((journal_main['article_df'][journal_id].iloc[article_id,0],journal_main['article_df'][journal_id].iloc[article_id,1],article_id,journal_id))
        # print(name,url)
    return l
user_input = "AI is my favourite language in 2022"
l=get_links(user_input)
print(l)

[(' language test activism', 'http://link.springer.com/article/10.1007/s10993-022-09614-7'), (' language advocacy times securitization network languagerights', 'http://link.springer.com/article/10.1007/s10993-022-09617-4'), (' attribution autonomy role robotic language acquisition', 'http://link.springer.com/article/10.1007/s00146-020-01114-8'), (' auditing large language approach', 'http://link.springer.com/article/10.1007/s43681-023-00289-2'), (' officiality strategic ambiguity language exploring migrant experiences andorra luxembourg', 'http://link.springer.com/article/10.1007/s10993-021-09602-3'), (' bias comparison framework abusive language datasets', 'http://link.springer.com/article/10.1007/s43681-021-00081-0'), (' ai management beyond exploring ai organizational context', 'http://link.springer.com/article/10.1007/s00146-021-01249-2'), (' automated occupation coding hierarchical approach classification language models', 'http://link.springer.com/article/10.1007/s44163-023-00050